# L3 Switch/Router Controller

The following guides through the process of creating a L3 switch/router using Ryu and OpenFlow

#### A few key points:
* By default Ryu doesn't implement any mechanisms to handle IP packet handling. 
* Ryu also doesn't implement ARP handling. (Which is needed for IP packet forwarding)
* Read l2_switch_controller notebook before continuing as I reuse code from there

Before continuing with the implementation, here's a quick summary of IP routing:
* Suppose you initiate a ping request in a host.
* If the destination is in the same subnet, it directly sends it to the destination.
* Otherwise, it sends it to the gateway router. 
* The IP address of the destination/gateway router is already known using protocols like DNS/DHCP.
* The host then constructs the IP packet with the appropriate src and dst ip addresses.
* Now it needs to construct the ethernet frame to send it over the wire.
* In order to construct a ethernet frame, the host needs the destination mac address.
* It checks its arp cache to see if it exists. If not, ARP broadcast request (destination mac is set to FF:FF:FF:FF:FF:FF and destination ip is set to the target ip) is sent to discover the mac address of the destination. The host also includes its own mac address in the request.
* Once the gateway router/destination host receives an arp request with broadcast request, it checks the destination IP and if it matches its own IP, a unicast response with the actual mac address is sent back.
* Now the host can construct an ethernet frame with proper mac.

Here is a video explaining the same: https://www.youtube.com/watch?v=QPi5Nvxaosw

We now define the same boilerplate code from the previous notebook:

In [ ]:
from ryu.base import app_manager
from ryu.controller import ofp_event
from ryu.controller.handler import CONFIG_DISPATCHER, MAIN_DISPATCHER
from ryu.controller.handler import set_ev_cls
from ryu.ofproto import ofproto_v1_3
from ryu.lib.packet import packet, ethernet, ether_types, ipv4, arp
import ipaddress

In [ ]:
class L3SwitchController(app_manager.RyuApp):
    OFP_VERSIONS = [ofproto_v1_3.OFP_VERSION]

    def __init__(self, *args, **kwargs):
        super(LearningSwitch, self).__init__(*args, **kwargs)
        self.switch_forwarding_table = {}

    @set_ev_cls(ofp_event.EventOFPSwitchFeatures, CONFIG_DISPATCHER)
    def switch_features_handler(self, ev):
        
        datapath = ev.msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        # Initial flow entry for matching misses
        match = parser.OFPMatch()
        actions = [parser.OFPActionOutput(ofproto.OFPP_CONTROLLER,
                                          ofproto.OFPCML_NO_BUFFER)]
        self.add_flow(datapath, 0, match, actions)

    def add_flow(self, datapath, priority, match, actions):
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        inst = [parser.OFPInstructionActions(ofproto.OFPIT_APPLY_ACTIONS, actions)]
        mod = parser.OFPFlowMod(datapath=datapath, priority=priority,
                                match=match, instructions=inst)
        datapath.send_msg(mod)

Now, in the handler, we parse specifically the ARP packet and then process it.

In [ ]:
@set_ev_cls(ofp_event.EventOFPPacketIn, MAIN_DISPATCHER)
def _packet_in_handler(self, ev):
    msg = ev.msg
    datapath = msg.datapath # datapath is the switch which got the packet

    in_port = msg.match['in_port']

    pkt = packet.Packet(msg.data)

    arp_pkt = pkt.get_protocol(arp.arp)
    ip_pkt = pkt.get_protocol(ipv4.ipv4)
    
    if arp_pkt:
        if arp_pkt.opcode == arp.ARP_REQUEST: 
            self.send_arp_response(datapath, arp_pkt, in_port)

    if ip_pkt:
        self.route_ip_packet(datapath, msg, ip_pkt, in_port)

In [1]:
def send_arp_response(self, datapath, arp_pkt, in_port):        
    actions = [datapath.ofproto_parser.OFPActionOutput(in_port)]

    src_mac = get_mac_for_switch(datapath.id) # could be random, doesn't matter (refer: https://github.com/mininet/mininet/wiki/FAQ#assign-macs)

    e = ethernet.ethernet(
            dst = arp_pkt.src_mac, # sending the response back to the source
            src = src_mac, 
            ethertype = ether_types.ETH_TYPE_ARP
        )
    
    a = arp.arp(
            opcode = arp.ARP_REPLY,
            dst_mac = arp_pkt.src_mac, # sending the response back to the source
            dst_ip = arp_pkt.src_ip,
            src_mac = src_mac,
            src_ip = arp_pkt.dst_ip
        )

    p = packet.Packet()
    p.add_protocol(e)
    p.add_protocol(a)
    p.serialize()

    out = datapath.ofproto_parser.OFPPacketOut(
            datapath=datapath,
            buffer_id=0xffffffff, # ensures no buffer is set
            in_port=datapath.ofproto.OFPP_CONTROLLER,
            actions=actions,
            data=p.data
    )
    
    datapath.send_msg(out)

The above function defines a simple ARP response handler. We obtain the request and construct an ARP response packet, serialize it along with an ethernet frame and send it back to the source

Now that we have the ARP setup, all we need next is to simply forward the IP packet to the appropriate port. Here is the spec for IP forwarding: https://datatracker.ietf.org/doc/html/rfc1812#page-85

* Basically, we send the packet to the appropriate port by looking at the routing table.
* If there is no entry in the table, we drop it.
* If there are multiple entries, we sort based on tos.
* We also decrement the TTL.

In [ ]:
def route_ip_packet(self, datapath, msg, ip_pkt, in_port):
    available_routes = self.routing_table.items()

    # The following router implementation is based on RFC 1812 (https://datatracker.ietf.org/doc/html/rfc1812#page-85)

    # The router located for matching routes in its routing table for the given ip address
    try:
        port_to_route  = self.routing_table[datapath.id][ip_pkt.dst]
    except:
        self.logger.info("route not found")

    if msg.buffer_id == datapath.of_proto.OFP_NO_BUFFER:
        data = msg.data
    
    if data is None:
        return

    actions = [datapath.parser.OFPActionOutput(port_to_route)]
    
    out = datapath.parser.OFPPacketOut(
        datapath=datapath, buffer_id=msg.buffer_id, in_port=in_port,
        actions=actions, data=data)

    datapath.send_msg(out)
    

#### And that's it! A few points:
* For IP routing, we do not learn the routes while handling packets.
* Routes are assigned statically in the routing table during the start or they are learned using dynamic routing protocols (DRP)
* The most common DRP is OSPF (which we will use in the next notebook)
* The python version is in controllers/l3_switch_controller.py
* Continue reading the fat_tree_topology notebook to learn more about the next steps